In [27]:
# imports
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler

In [31]:
# One-hot encode
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    dummies = dummies.iloc[:, :-1]
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return res
    

In [32]:
# Reading data
telco_data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv',
                         converters={
                             'gender': lambda x: int(x == 'Female'),
                             'Partner': lambda x: int(x == 'Yes'),
                             'Dependents': lambda x: int(x == 'Yes'),
                             'PhoneService': lambda x: int(x =='Yes'),
                             'MultipleLines': lambda x: int(x == 'Yes'),
                             'OnlineSecurity': lambda x: int(x == 'Yes'),
                             'OnlineBackup': lambda x: int(x == 'Yes'),
                             'DeviceProtection': lambda x: int(x == 'Yes'),
                             'TechSupport': lambda x: int(x == 'Yes'),
                             'StreamingTV': lambda x: int(x == 'Yes'),
                             'StreamingMovies': lambda x: int(x == 'Yes'),
                             'PaperlessBilling': lambda x: int(x =='Yes'),
                             'Churn': lambda x: int(x =='Yes'),
                             'MonthlyCharges': lambda x: float(x)
                         })

In [33]:
# Preprocessing
telco_data.drop('customerID', axis=1, inplace=True)
telco_data = telco_data.astype({
    'tenure': int,
    "MonthlyCharges": float,
    "TotalCharges": float
}, errors="ignore")

totalChargesMedian = (telco_data['TotalCharges'].loc[telco_data['TotalCharges'] != ' ']).median()
telco_data['TotalCharges'].replace([' '], totalChargesMedian, regex=True, inplace=True)

columns_to_encode = ['InternetService', 'Contract', 'PaymentMethod']
for column in columns_to_encode:
    telco_data = encode_and_bind(telco_data, column)

# Move final column for better visualization
telco_data.insert(len(telco_data.columns)-1, 'Churn', telco_data.pop('Churn'))

all_columns = list(telco_data.columns)
telco_data[all_columns] = MinMaxScaler().fit_transform(telco_data[all_columns])

/tmp/ipykernel_55997/2583104947.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  telco_data.InternetService_No = telco_data.InternetService_DSL + telco_data['InternetService_Fiber optic']


In [ ]:
display(telco_data.iloc[487:494, :])
telco_data.to_csv('telco.csv')

In [ ]:
print(telco_data.shape)

